### Задание 1
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156. Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик 20000 человек в день. Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .

In [23]:
from typing import Union
from statsmodels.stats.power import tt_ind_solve_power
import math

# Создаем функцию для расчета размера эффекта между двумя выборками
# с непрерывными признаками


def continuous_effect_size(mean1: Union[float, int],
                           std1: Union[float, int],
                           mean2: Union[float, int],
                           std2: Union[float, int],
                           nobs1: int = 10_000,
                           nobs2: int = 10_000) -> float:

    effect_size_formula = (mean1 - mean2) / ((std1**2 + std2**2) / 2) ** 0.5
    return effect_size_formula

# Создаем функцию для расчета необходимого размера выборок,
# требуемой статистической мощности и уровня значимости


def sample_size_continuous(effect_size: float,
                           alpha: float = .05,
                           beta: float = .2,
                           ratio: Union[float, int] = 1) -> int:

    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                           )
    return int(n * 2)


# Задаем параметры эксперимента (изменяем среднее и стандартное отклонение выборки
# теста на 10%)
mu_control, mu_test = 25, 25*1.10
std_control, std_test = 156, 156*1.10
traffic_per_day = 20000

# Вычисляем размер эффекта между выборками
effect_size = continuous_effect_size(
    mean1=mu_control, std1=std_control, mean2=mu_test, std2=std_test)

# Расчитываем размер выборки с требуемой статистической мощностью на заданном
# уровне значимости. Определяем, сколько дней нужно для получения достаточного
# трафика на сайте
traffic = sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)
traffic_per_day = 20000
traffic_per_day_needed = traffic / traffic_per_day

print(traffic)
print(math.ceil(traffic_per_day_needed))



135084
7


Ответ:  7 дней необходимо для эксперимента, при его реализации на всех 135084 пользователях, при alpha = 5% и beta = 20%.

### Задание 2
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта. Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.Средний трафик 4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%. Калькуляторы, которые могут помочь в решение задачь: https://www.evanmiller.org/ab-testing/sample-size.html и https://www.statulator.com/SampleSize/ss2M.html

In [42]:
# импорт необходимых библиотек
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from typing import Union
from math import asin
import numpy as np

# Создаем функцию для расчета эффекта между двумя долями конверсии

def calc_proportion_effect_size(conv_1: int, nobs_1: int, conv_2: int, nobs_2: int):
    prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
    
    es_formula = 2  * asin(np.sqrt(prob_1)) - 2 * asin(np.sqrt(prob_2))
    es_import = proportion_effectsize(prob_1, prob_2)
    
    return es_formula, es_import

# Создаем функцию для расчета необходимого размера выборки


def calc_proportion_sample_size(effect_size: float,
                                alpha: float = 0.05,
                                beta: float = 0.2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)




Метрика bounce rate в GA = 40%.
При оптимизации сайта она изменится минимум на 20% от текущего показателя. =>
40% х 0,2 = 8%
40% - 8% = 32% =>
H0=40% # bounce rate в GA
H1=32% # bounce rate в GA

Найдем размер выборки для каждой гипотезы, при помощи калькулятора https://www.evanmiller.org/ab-testing/sample-size.html

![Image alt](https://github.com/Chernaya-Nataliya/AB-Testing/blob/main/6.1.png)

![Image alt](https://github.com/Chernaya-Nataliya/AB-Testing/blob/main/6.2.png)

In [43]:

# Расчитываем effect_size.
conv_1, conv_2 = 40, 32
nobs_1, nobs_2 = 983, 983
prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
calc_proportion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)

# Расчитываем минимально необходимую выборку
effect_size = calc_proportion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)[0]
sample_size = calc_proportion_sample_size(effect_size=effect_size, alpha=.05, beta=0.2)

print(f"Общий размер выборки {sample_size} пользователя.")


# Расчитаем количество дней.
days_traffic = math.ceil(sample_size/4000)

print(f"Необходимо провести эксперимент в течение {days_traffic} дней")

Общий размер выборки 16673 пользователя.
Необходимо провести эксперимент в течение 5 дней


Ответ:  5 дней необходимо для эксперимента, при его реализации на всех 16673 пользователях, при alpha = 5% и beta = 20%.